# 6.4 Prioritized Experience Replay
Q学習がきちんと進んでない状態`s`のtransitionに対して、
優先的に学習させる深層強化学習。
Prioritizeとは、プライオリティー(優先順位)をつけるという意味。

次の式で表される価値関数のベルマン方程式の絶対値誤差(TD誤差)を元に優先順位をつける。

$$
|[R(t+1)+\gamma \times \max_a[Q(s(t+1),a)]] - Q(s(t), a(t))|
$$

このTD誤差が大きいtransitionを優先的にExperience Replay時に学習させ、価値関数のネットワークの
出力誤差が小さくなるようにする。

In [ ]:
import numpy as np

TD_ERROR_EPSILON = 0.0001

class TDErrorMemory:
    
    def __init__(self, CAPACITY):
        self.capacity = CAPACITY
        self.memory = []
        self.index = 0
        
    def push(self, td_error):
        """TD誤差をメモリに保存"""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
            
        self.memory[self.index] = td_error
        self.index = (self.index + 1) % self.capacity
        
    def __len__(self):
        return len(self.memory)
    
    def get_prioritized_indexes(self, batch_size):
        """TD誤差に応じた確率でindexを取得"""
        sum_absolute_td_error = np.sum(np.absolute(self.memory))
        sum_absolute_td_error += TD_ERROR_EPSILON * len(self.memory)
        
        # batch_size分の乱数を生成して、昇順にする
        rand_list = np.random.uniform(0, sum_absolute_td_error, batch_size)
        rand_list = np.sort(rand_list)
        
        indexes = []
        idx = 0
        tmp_sum_absolute_td_error = 0
        for rand_num in rand_list:
            while tmp_sum_absolute_td_error < rand_num:
                tmp_sum_absolute_td_error += (
                    abs(self.memory[idx]) + TD_ERROR_EPSILON
                )
                idx += 1
            
            if idx >= len(self.memory):
                idx = len(self.memory) - 1
            indexes.append(idx)
            
        return indexes
    
    def update_td_error(self, updated_td_errors):
        """TD誤差の更新"""
        self.memory = updated_td_errors
            